In [2]:


import os
import time
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance,
    VectorParams,
    PointStruct,
    Filter,
)
from sentence_transformers import SentenceTransformer


DB_PATH = "./qdrant_storage"

# Убеждаемся, что директория для базы данных существует.
# Если директория не существует, она будет создана. Это предотвращает ошибки
# при попытке Qdrant записать данные в несуществующее место.
os.makedirs(DB_PATH, exist_ok=True)

# Инициализируем QdrantClient с обработкой ошибок подключения
max_retries = 3
retry_delay = 2  # seconds
client = None

for attempt in range(max_retries):
    try:
        # Закрываем все существующие соединения клиента, если они есть
        if client is not None:
            try:
                client.close()
            except:
                pass

        # Создаем нового клиента с новым соединением
        client = QdrantClient(
            path=DB_PATH
        )
        # Проверяем соединение, чтобы убедиться, что оно работает
        collections = client.get_collections()
        break  # Успех - выходим из цикла повторных попыток
    except Exception as e:
        if attempt < max_retries - 1:
            print(f"Qdrant connection attempt {attempt + 1} failed: {e}. Retrying in {retry_delay}s...")
            time.sleep(retry_delay)
            retry_delay *= 2  # Экспоненциальная задержка
        else:
            print(f"Не удалось подключиться к Qdrant после {max_retries} попыток: {e}")
            raise

print(f"QdrantClient инициализирован в локальном режиме. Данные будут храниться в: {DB_PATH}")

/home/nikita/PROJECTS/Pydantic-ai/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


QdrantClient инициализирован в локальном режиме. Данные будут храниться в: ./qdrant_storage


In [3]:
from sentence_transformers import SentenceTransformer
import torch


# Определяем путь для кэширования модели эмбеддингов.
# Это позволяет избежать повторной загрузки модели при каждом запуске.
MODEL_CACHE_FOLDER = "./models/frida"
os.makedirs(MODEL_CACHE_FOLDER, exist_ok=True)

# Загружаем модель ai-forever/FRIDA.
# cache_folder: Указывает директорию для сохранения загруженных файлов модели.
# trust_remote_code=True: Этот параметр необходим для некоторых моделей из Hugging Face Hub,
# которые содержат пользовательский код в своих репозиториях. Его установка в True
# разрешает выполнение этого удаленного кода на локальной машине.
# Важно отметить, что этот параметр следует использовать с осторожностью и только для
# моделей из доверенных источников, так как он может представлять потенциальный риск безопасности,
# если код не был проверен.[3, 4]
try:
    encoder = SentenceTransformer("ai-forever/FRIDA", cache_folder=MODEL_CACHE_FOLDER, trust_remote_code=True)
    print(f"Модель ai-forever/FRIDA успешно загружена и кэширована в: {MODEL_CACHE_FOLDER}")

    # Проверяем наличие GPU (CUDA) и перемещаем модель на соответствующее устройство для ускорения вычислений,
    # если GPU доступен. Это значительно повышает производительность при генерации эмбеддингов,
    # особенно для больших объемов текста.[4]
    device = "cuda" if torch.cuda.is_available() else "cpu"
    encoder.to(device)
    print(f"Модель FRIDA использует устройство: {device}")

except Exception as e:
    print(f"Ошибка при загрузке модели: {e}")

Модель ai-forever/FRIDA успешно загружена и кэширована в: ./models/frida
Модель FRIDA использует устройство: cuda


In [4]:
texts = ["Привет, мир!", "Как дела?", "Это тестовое предложение."]
embeddings = encoder.encode(texts, 
                            convert_to_tensor=False, 
                            show_progress_bar=True, 
                            normalize_embeddings=True,
                            prompt="search_documents")
print(f"Размерность сгенерированных эмбеддингов: {embeddings.shape}")
print(f"Размерность одного эмбеддинга: {encoder.get_sentence_embedding_dimension()}")

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.29it/s]

Размерность сгенерированных эмбеддингов: (3, 1536)
Размерность одного эмбеддинга: 1536


**При выборе префикса мы используем следующие основные правила:**

- "search_query: "и "search_document: "префиксы предназначены для ответа или поиска соответствующего абзаца
- "paraphrase: "префикс для задач, связанных с симметричным перефразированием (STS, анализ парафраз, дедупликация)
- "categorize: "префикс предназначен для асимметричного сопоставления заголовка и текста документа (например, новости, научные статьи, социальные посты)
- "categorize_sentiment: "префикс используется для любых задач, которые зависят от особенностей настроений - (например, ненависть, токсичность, эмоции)
- "categorize_topic: "префикс предназначен для задач, где необходимо сгруппировать тексты по темам
- "categorize_entailment: "префикс для текстовой задачи вывода (NLI)

In [5]:
type(embeddings)

numpy.ndarray

**Создание новой коллекции**

Для создания новой коллекции необходимо указать ее имя и конфигурацию векторов, 
включая размерность (количество измерений) и метрику расстояния, 
которая будет использоваться для измерения сходства между векторами. 
Выбор метрики расстояния (например, косинусное сходство, скалярное произведение, евклидово расстояние) зависит от того, как была обучена модель эмбеддингов.

In [6]:
COLLECTION_NAME = "my_first_collection"

# Создаем коллекцию в Qdrant, если она еще не существует.
if not client.collection_exists(COLLECTION_NAME):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(
            size=encoder.get_sentence_embedding_dimension(), # Размерность эмбеддингов
            distance=Distance.COSINE # Используем косинусное расстояние для сравнения векторов
        )
    )
    print(f"Коллекция '{COLLECTION_NAME}' успешно создана.")
else:
    print(f"Коллекция '{COLLECTION_NAME}' уже существует. Пропускаем создание.")

Коллекция 'my_first_collection' уже существует. Пропускаем создание.


### [ссылка](https://api.qdrant.tech/v-1-12-x/api-reference/collections/get-collections) на апи qdrant

In [7]:
collections = client.get_collections()
print("Существующие коллекции в Qdrant:")
for collection in collections.collections:
    print(f"- {collection.name}")

Существующие коллекции в Qdrant:
- cooking
- my_first_collection


In [8]:
if client.collection_exists(collection_name=COLLECTION_NAME):
    collection_info = client.get_collection(collection_name=COLLECTION_NAME)
    print(f"\nДетали коллекции '{COLLECTION_NAME}':")
    print(f"  Статус: {collection_info.status}")
    print(f"  Количество точек: {collection_info.points_count}")
    print(f"  Конфигурация векторов: {collection_info.config}")
    print(f"  Схема полезной нагрузки (индексы): {collection_info.payload_schema}")
else:
    print(f"\nКоллекция '{COLLECTION_NAME}' не найдена.")


Детали коллекции 'my_first_collection':
  Статус: green
  Количество точек: 2
  Конфигурация векторов: params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None) hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None) optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1) wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0) quantization_config=None strict_mode_config=None
  Схема полезной нагрузки (индексы): {}


Получение информации о payload_schema особенно важно, так как оно показывает, 
какие поля полезной нагрузки были проиндексированы. 
Это знание критически важно для эффективной фильтрации, поскольку фильтрация по проиндексированным полям значительно быстрее. [ccылка](https://qdrant.tech/documentation/database-tutorials/automate-filtering-with-llms/)

**Редактирование конфигурации коллекции**

Qdrant позволяет обновлять конфигурацию существующей коллекции, например, 
изменять параметры HNSW индекса, настройки оптимизаторов или параметры квантования. 
Это может быть полезно для тонкой настройки производительности коллекции после ее создания.
[Документация](https://python-client.qdrant.tech/qdrant_client.qdrant_client)

In [9]:
from qdrant_client.models import HnswConfigDiff

# Обновляем параметры HNSW для коллекции.
# m: определяет количество связей на узел в графе HNSW. Более высокое значение увеличивает точность поиска,
# но требует больше памяти и времени на построение индекса.
# ef_construct: контролирует размер диапазона поиска во время построения индекса. Большее значение
# улучшает точность индекса, но увеличивает время построения.[10]
try:
    client.update_collection(
        collection_name=COLLECTION_NAME,
        hnsw_config=HnswConfigDiff(
            m=16, # Уменьшено для примера, по умолчанию может быть 16 или 32
            ef_construct=100 # По умолчанию 100
        )
    )
    print(f"\nКонфигурация HNSW для коллекции '{COLLECTION_NAME}' обновлена.")
except Exception as e:
    print(f"\nОшибка при обновлении конфигурации коллекции: {e}")


Конфигурация HNSW для коллекции 'my_first_collection' обновлена.


[Статья из документации](https://qdrant.tech/articles/vector-search-resource-optimization/)

**Удаление коллекции**

Удаление коллекции полностью очищает все данные и связанную с ней конфигурацию. 
Это необратимая операция, поэтому следует использовать ее с осторожностью.

In [10]:
# Удаление коллекции для очистки среды после примеров.
if client.collection_exists(collection_name=COLLECTION_NAME):
    client.delete_collection(collection_name=COLLECTION_NAME)
    print(f"\nКоллекция '{COLLECTION_NAME}' успешно удалена.")
else:
    print(f"\nКоллекция '{COLLECTION_NAME}' не существует, удаление не требуется.")


Коллекция 'my_first_collection' успешно удалена.


**Работа с Точками (Векторами и Полезной Нагрузкой)**

Точки являются центральными сущностями, с которыми оперирует Qdrant. 
Каждая точка состоит из вектора (или нескольких векторов) и необязательной полезной нагрузки (payload), 
которая представляет собой произвольные метаданные, связанные с вектором.   



In [11]:
# Пример текстовых данных для индексации
documents = [
    {"id": 1,
     "text": "Привет, мир!",
     "category": "greeting"},
    {"id": 2,
     "text": "Как дела?",
     "category": "greeting"},
    {"id": 3,
     "text": "Это тестовое предложение.",
     "category": "test"}
]

# Генерируем эмбеддинги для текстовых данных
# convert_to_tensor=False, так как Qdrant ожидает список Python, а не тензор PyTorch.
document_vectors = encoder.encode([doc["text"] for doc in documents], convert_to_tensor=False,
                                   show_progress_bar=True, 
                                   normalize_embeddings=True,
                                   prompt="search_documents").tolist()

print(f"\nСгенерировано {len(document_vectors)} эмбеддингов.")

Batches: 100%|██████████| 1/1 [00:00<00:00, 116.72it/s]


Сгенерировано 3 эмбеддингов.


**Добавление и обновление точек (Upserting points)**

Операция upsert в Qdrant позволяет одновременно вставлять новые точки и обновлять существующие.

Если точка с указанным ID уже существует, она будет перезаписана;

в противном случае будет создана новая точка. 

Это делает операции идемпотентными, что упрощает управление данными.   

In [12]:
# Создаем коллекцию, если она была удалена в предыдущем примере
if not client.collection_exists(collection_name=COLLECTION_NAME):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=encoder.get_sentence_embedding_dimension(), 
                                    distance=Distance.COSINE),
    )
    print(f"Коллекция '{COLLECTION_NAME}' создана для добавления точек.")

# Подготавливаем точки для upsert
points_to_upsert = []
for doc, vector in zip(documents, document_vectors):
    points_to_upsert.append(
        PointStruct(
            id=doc["id"],
            vector=vector,
            payload={
                "text": doc["text"],
                "category": doc["category"]
            }
        )
    )
# Выполняем массовую загрузку (upsert) точек
operation_info = client.upsert(
    collection_name=COLLECTION_NAME,
    wait=True, # Ожидаем завершения операции
    points=points_to_upsert
)
print(f"\nОперация upsert завершена: {operation_info.status}. Количество точек: {len(points_to_upsert)}")

# Добавляем новую точку или обновляем существующую
new_document = {"id": 9, "text": "Пример новой точки для демонстрации обновления.", "category": "new_data"}
new_vector = encoder.encode([new_document["text"]], convert_to_tensor=False).tolist()

client.upsert(
    collection_name=COLLECTION_NAME,
    wait=True,
    points=[
        PointStruct(
            id=new_document["id"],
            vector=new_vector,
            payload={
                "text": new_document["text"],
                "category": new_document["category"]
            }
        )
    ]
)
print(f"Точка с ID {new_document['id']} добавлена/обновлена.")

Коллекция 'my_first_collection' создана для добавления точек.

Операция upsert завершена: completed. Количество точек: 3
Точка с ID 9 добавлена/обновлена.


**Существует метод получения точек по их идентификаторам.***

In [13]:
client.retrieve(
    collection_name=COLLECTION_NAME,
    ids=[2],
    with_payload=True,
    # with_vectors=True,
)

[Record(id=2, payload={'text': 'Как дела?', 'category': 'greeting'}, vector=None, shard_key=None, order_value=None)]

**Фильрация по payload \ metadata**

Vожет возникнуть необходимость получить все сохраненные точки, не зная идентификаторов, 

или выполнить итерацию по точкам, соответствующим фильтру.

[Документация](https://qdrant.tech/documentation/concepts/points/?q=retrieve#retrieve-points)

In [14]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

client.scroll(
    collection_name=COLLECTION_NAME,
    scroll_filter=Filter(
        must=[
            FieldCondition(key="category", match=MatchValue(value="greeting")),
        ]
    ),
    # limit=2,
    with_payload=True,
    with_vectors=False,
)

([Record(id=1, payload={'text': 'Привет, мир!', 'category': 'greeting'}, vector=None, shard_key=None, order_value=None),
  Record(id=2, payload={'text': 'Как дела?', 'category': 'greeting'}, vector=None, shard_key=None, order_value=None)],
 None)

**Удаление точек**

по ID

In [15]:
from qdrant_client.models import PointIdsList

# Удаление точки по ID
client.delete(
    collection_name=COLLECTION_NAME,
    points_selector=PointIdsList(
        points=[1,],  # ID точки для удаления
    )
)
client.retrieve(
    collection_name=COLLECTION_NAME,
    ids=[1],
    with_payload=True,
    # with_vectors=True,
)

[]

По фильтру

In [16]:
client.delete(
    collection_name=COLLECTION_NAME,
    points_selector=Filter(
        must=[
            FieldCondition(
                key="category", 
                match=MatchValue(value="new_data")),
        ]
    )
)
print("Все точки с категорией 'new_data' удалены.")

# Проверяем оставшиеся точки
remaining_points = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=10,
    with_payload=True
)
print("\nОставшиеся точки в коллекции:")
print(remaining_points)

Все точки с категорией 'new_data' удалены.

Оставшиеся точки в коллекции:
([Record(id=2, payload={'text': 'Как дела?', 'category': 'greeting'}, vector=None, shard_key=None, order_value=None), Record(id=3, payload={'text': 'Это тестовое предложение.', 'category': 'test'}, vector=None, shard_key=None, order_value=None)], None)


Удаление по фильтру является мощным инструментом для управления данными, позволяя массово удалять точки, 

соответствующие определенным критериям, без необходимости предварительного получения их ID.  

**Подсчет точек в коллекции**

Для получения количества точек в коллекции, соответствующих определенному фильтру или всех точек, 

используется метод count(). Этот метод позволяет получить точное или приблизительное количество, 

причем приблизительный подсчет выполняется быстрее.

In [17]:
from qdrant_client.models import Range

total_points_count = client.count(
    collection_name=COLLECTION_NAME,
    exact=True,
)
print(f"\nОбщее количество точек в коллекции '{COLLECTION_NAME}': {total_points_count}")


Общее количество точек в коллекции 'my_first_collection': count=2


In [18]:
remaining_points = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=10,
    with_payload=True
)
remaining_points

([Record(id=2, payload={'text': 'Как дела?', 'category': 'greeting'}, vector=None, shard_key=None, order_value=None),
  Record(id=3, payload={'text': 'Это тестовое предложение.', 'category': 'test'}, vector=None, shard_key=None, order_value=None)],
 None)

In [19]:
# Подсчет точек, соответствующих определенному фильтру 
filtered_points_count = client.count(
    collection_name=COLLECTION_NAME,
    count_filter=Filter(
        must=[
            FieldCondition(
                key="category", 
                match=MatchValue(value="greeting")),
        ]
    ),
    exact=True
)
print(f"Количество точек с категорией 'greeting': {filtered_points_count.count}")

Количество точек с категорией 'greeting': 1


### Векторный Поиск (Vector Search)

Векторный поиск, или поиск ближайших соседей (k-NN), является основной функцией Qdrant. 

Он позволяет находить векторы, наиболее похожие на заданный запрос, в многомерном пространстве. 

Сходство определяется выбранной метрикой расстояния.

In [69]:
# Повторно создадим коллекцию и добавим данные, если они были удалены
if not client.collection_exists(collection_name=COLLECTION_NAME):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE),
    )
points_to_upsert =[
    PointStruct(
        id=doc["id"],
        vector=vector,
        payload={
            "text": doc["text"],
            "category": doc["category"]
        }
    )
    for doc, vector in zip(documents, document_vectors)
]


# Выполняем массовую загрузку (upsert) точек
operation_info = client.upsert(
    collection_name=COLLECTION_NAME,
    wait=True, # Ожидаем завершения операции
    points=points_to_upsert
    )

print(f"\nКоллекция '{COLLECTION_NAME}' создана и заполнена для векторного поиска.")


Коллекция 'my_first_collection' создана и заполнена для векторного поиска.


In [70]:
client.count(
    collection_name=COLLECTION_NAME,
    exact=True, # Точный подсчет точек в коллекции
)

CountResult(count=3)

**Базовый поиск**

In [71]:
query_text = "как дела?"
query_vector = encoder.encode(query_text, convert_to_tensor=False, normalize_embeddings=True, prompt="search_query: ") # .tolist()[0]

hits = client.query_points(
    collection_name=COLLECTION_NAME,
    query=query_vector,
    limit=3,  # Ограничиваем количество результатов
    with_payload=True,  # Возвращаем полезную нагрузку (payload) для каждого результата
    with_vectors=False,  # Не возвращаем векторы, только полезную нагрузку
).points
print(f"\nРезультаты поиска для запроса '{query_text}':")
for hit in hits:
    if hit:
        print(f"ID: {hit.id}, Text: {hit.payload['text']}, Category: {hit.payload['category']}, Score: {hit.score}")



Результаты поиска для запроса 'как дела?':
ID: 2, Text: Как дела?, Category: greeting, Score: 0.8052579267936519
ID: 1, Text: Привет, мир!, Category: greeting, Score: 0.2915120095537881
ID: 3, Text: Это тестовое предложение., Category: test, Score: 0.1939931447475377


In [72]:
hits

[ScoredPoint(id=2, version=0, score=0.8052579267936519, payload={'text': 'Как дела?', 'category': 'greeting'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1, version=0, score=0.2915120095537881, payload={'text': 'Привет, мир!', 'category': 'greeting'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=3, version=0, score=0.1939931447475377, payload={'text': 'Это тестовое предложение.', 'category': 'test'}, vector=None, shard_key=None, order_value=None)]

Параметр score_threshold позволяет отфильтровать результаты поиска, которые имеют низкий балл сходства, 

что полезно для исключения нерелевантных результатов. Поведение score_threshold зависит от используемой метрики расстояния: 

для косинусного сходства исключаются результаты с баллом ниже порога, а для евклидова расстояния — выше порога.
**[Документация](https://qdrant.tech/documentation/concepts/search/)

In [73]:
hits_with_trashold = client.query_points(
    collection_name=COLLECTION_NAME,
    query=query_vector,
    score_threshold=0.5,  # Устанавливаем порог для отбора результатов
).points
print(f"\nРезультаты поиска с порогом 0.5 для запроса '{query_text}':")
for hit in hits_with_trashold:
    print(f"ID: {hit.id}, Text: {hit.payload['text']}, Category: {hit.payload['category']}, Score: {hit.score}")


Результаты поиска с порогом 0.5 для запроса 'как дела?':
ID: 2, Text: Как дела?, Category: greeting, Score: 0.8052579267936519


**Настройка параметров HNSW для оптимизации**

HNSW (Hierarchical Navigable Small World) — это алгоритм индексации, используемый Qdrant для эффективного векторного поиска. 

Параметры HNSW, такие как m и ef_construct (при создании/обновлении коллекции), а также hnsw_ef (при выполнении запроса), влияют на баланс между скоростью поиска, точностью и потреблением памяти.   

m: Максимальное количество связей для каждого узла в графе HNSW. Более высокое значение m увеличивает точность поиска, но также требует больше памяти и времени на построение индекса.

ef_construct: Размер списка ближайших соседей, который рассматривается во время построения индекса. Большее значение ef_construct улучшает качество индекса, но увеличивает время его построения.

hnsw_ef: Размер списка ближайших соседей, который рассматривается во время выполнения запроса. 
Большее значение hnsw_ef повышает точность поиска, но увеличивает время ответа на запрос.   


In [80]:
from qdrant_client.models import SearchParams
# Пример поиска с настроенным параметром hnsw_ef
# Этот параметр влияет на точность и скорость поиска
hits_optimized = client.query_points(
    collection_name=COLLECTION_NAME,
    query=query_vector,
    limit=3,
    search_params=SearchParams(
        hnsw_ef=256, 
        exact=True), # Увеличиваем ef для потенциально лучшей точности
    with_payload=True
).points

print(f"\nРезультаты векторного поиска с SearchParams(hnsw_ef=128) для запроса '{query_text}':")
for hit in hits_optimized:
    print(f"- ID: {hit.id}, Score: {hit.score:.4f}, Text: '{hit.payload.get('text')}'")


Результаты векторного поиска с SearchParams(hnsw_ef=128) для запроса 'как дела?':
- ID: 2, Score: 0.8053, Text: 'Как дела?'
- ID: 1, Score: 0.2915, Text: 'Привет, мир!'
- ID: 3, Score: 0.1940, Text: 'Это тестовое предложение.'
